In [11]:
import sys
print(sys.path)


['c:\\Users\\eys63\\Desktop\\기타활동\\2024\\겨울방학\\24Winter_Vacation\\25_01_03', 'c:\\Users\\eys63\\anaconda3\\python312.zip', 'c:\\Users\\eys63\\anaconda3\\DLLs', 'c:\\Users\\eys63\\anaconda3\\Lib', 'c:\\Users\\eys63\\anaconda3', '', 'C:\\Users\\eys63\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Users\\eys63\\anaconda3\\Lib\\site-packages', 'c:\\Users\\eys63\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\eys63\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\eys63\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\eys63\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor']


https://www.mss.go.kr/site/smba/ex/bbs/List.do?cbIdx=310 의 링크에서 1501번째 글의 첨부파일을 다운받았음. 이것을 활용할 예정임

In [ ]:
# pdf, 라이브러리 불러오기, 텍스트 추출
import time
from PyPDF2 import PdfReader
import fitz  # pymupdf
import pdfplumber
import pdfminer.high_level
import pypdfium2 as pdfium

# PDF 파일 경로
pdf_path = r"C:\Users\eys63\Desktop\기타활동\2024\겨울방학\24Winter_Vacation\data\2024년_중소기업_정책자금_융자공고(참고자료).pdf"

# PyPDF2로 텍스트 추출
def extract_with_pypdf(path):
    reader = PdfReader(path)
    return " ".join([page.extract_text() for page in reader.pages])

# PyMuPDF로 텍스트 추출
def extract_with_pymupdf(path):
    doc = fitz.open(path)
    return " ".join([page.get_text() for page in doc])

# PDFPlumber로 텍스트 추출
def extract_with_pdfplumber(path):
    with pdfplumber.open(path) as pdf:
        return " ".join([page.extract_text() for page in pdf.pages])

# PDFMiner로 텍스트 추출
def extract_with_pdfminer(path):
    return pdfminer.high_level.extract_text(path)

# PyPDFium2로 텍스트 추출
def extract_with_pypdfium2(path):
    pdf = pdfium.PdfDocument(path)
    return " ".join([pdf[page].get_textpage().get_text_range() for page in range(len(pdf))])


In [15]:
# 텍스트 분할
def split_text(text, chunk_size=500):
    words = text.split()
    return [words[i:i + chunk_size] for i in range(0, len(words), chunk_size)]


In [21]:
import config
import openai

# API 키 설정
openai.api_key = config.OPENAI_API_KEY


In [22]:
# OpenAI API를 사용한 평가 함수
def evaluate_with_openai(chunk_text):
    try:
        response = openai.Completion.create(
            model="gpt-4-mini",
            prompt=f"Evaluate the following text chunk and provide a score from 0 to 100, along with a brief explanation:\n\n{chunk_text}",
            max_tokens=150,
            n=1,
            stop=None,
            temperature=0.5,
        )
        content = response.choices[0].text.strip()
        score_line, explanation = content.split('\n', 1)
        score = int(score_line.split(':')[1].strip())
        return {"score": score, "explanation": explanation.strip()}
    except Exception as e:
        return {"error": str(e)}


In [23]:
# 평가

# PDF 텍스트 추출 (예: PyPDF2 사용)
text = extract_with_pypdf("")

# 텍스트를 500글자 단위로 나누기
chunks = split_text_by_chars(text, chunk_size=500)

# 각 조각 평가
results = []
for idx, chunk in enumerate(chunks):
    evaluation = evaluate_with_openai(chunk)
    evaluation["chunk_index"] = idx
    results.append(evaluation)

# 평가 결과 출력
for result in results:
    print(f"Chunk {result['chunk_index']} - Score: {result.get('score', 'Error')} - Explanation: {result.get('explanation', result.get('error'))}")


FileNotFoundError: [Errno 2] No such file or directory: 'your_pdf_file.pdf'